Đối với dữ liệu soil moisture 100m, chúng ta sẽ chọn điểm ngẫu nhiên trên các vùng có dữ liệu do Planet cung cấp. Phương pháp chọn như sau:
- Chia vùng dữ liệu thành các grid 1km, lọc và giữ lại các ô có phần lớn diện tích là crop, tree và grass.
- Với mỗi ô được giữ lại, chọn một điểm ngẫu nhiên trong đó. 

In [1]:
import ee
import geemap
ee.Initialize()

### Get Dynamic World V1 tif image over the ROI in Inidia and China

Đã tải về nên không cần chạy lại nữa các ảnh land cover được lưu trong các đường dẫn:
* India: training_data/100m/india/map/dw_india.tif
* China: training_data/100m/china/map/dw_china.tif

In [ ]:
import ee
ee.Initialize()

START = '2022-10-01'
END = '2022-10-31'

# Your region of interest
# geojson_path = 'china/map/china_grid.geojson'
# roi = geemap.geojson_to_ee(geojson_path)

# Chú ý cần thay đổi đường dẫn đến file geojson của bạn
roi = ee.FeatureCollection('projects/ee-lengocthanh/assets/china_grid').geometry()

dw = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
    .filterDate(START, END) \
    .filterBounds(roi) \
    .select('label') \
    .mode()

# Export as GeoTIFF
task = ee.batch.Export.image.toDrive(
    image=dw,
    description='dw_china',
    folder='gee_exports',
    fileNamePrefix='dw_mode_2022',
    region=roi.bounds(),
    scale=10,
    crs='EPSG:4326',
    maxPixels=1e13
)
task.start()


### Filter and retain grid cells meeting the requirements

In [1]:
import geopandas as gpd
import rasterio
import rasterio.mask
import numpy as np
import pandas as pd
root_path = "/mnt/data2tb/Transfer-DenseSM-E_pack/training_data/100m"
region = 'china'

# Load and convert your grid to EPSG:4326
grid = gpd.read_file(f"{root_path}/{region}/map/{region}_grid.gpkg").to_crs("EPSG:4326")

# Open the Dynamic World image (already downloaded as GeoTIFF)
with rasterio.open(f"{root_path}/{region}/map/dw_{region}.tif") as src:
    results = []

    for idx, row in grid.iterrows():
        geom = [row['geometry']]
        try:
            out_image, out_transform = rasterio.mask.mask(src, geom, crop=True)
            label_pixels = out_image[0]
            valid_pixels = label_pixels[label_pixels != src.nodata]
            
            if len(valid_pixels) == 0:
                continue
            
            # Count frequencies
            unique, counts = np.unique(valid_pixels, return_counts=True)
            label_count = dict(zip(unique, counts))

            # Sum up pixels for labels 1 (tree), 2 (grass), 4 (crop)
            selected_total = sum(label_count.get(k, 0) for k in [6])
            total = sum(counts)

            percent = selected_total / total * 100

            if percent > 30:
                results.append(row['id'])  # or use row.name, or any identifier

        except Exception as e:
            print(f"Failed on grid cell {row['id']}: {e}")

all_ids = grid['id']

# Create a new GeoDataFrame with only the selected grid cells
selected_grid = grid[grid['id'].isin(results)]

# Save to GPKG
selected_grid.to_file(f"{root_path}/{region}/map/filtered_grid/built_delete.gpkg", driver="GPKG")


### Không thể dùng code chung để chọn điểm lấy dữ liệu trên vùng Trung Quốc và Ấn Độ

Bởi vì có sự đặc biệt ở vùng lấy dữ liệu tại Trung Quốc. Ở đó đa số toàn đồi núi do đó, nếu lấy điểm ngẫu nhiên trong các grid cell được lọc sẽ thường rơi vào các vùng là rừng núi. Cách giải quyết đó là, xét trong từng grid cell, nếu tại ô đó có vùng crop sẽ chọn điểm ngẫu nhiên trên crop, còn không có crop trong ô, thì sẽ phải bắt buộc chọn điểm ngẫu nhiên trên vùng rừng núi.

Còn với Ấn Độ, vùng lấy dữ liệu là đồng bằng nên không cần phải làm phức tạp như vậy. Chỉ đơn giản là chọn điểm ngẫu nhiên trong từng grid cell

### Choose random points for India

In [ ]:
import geopandas as gpd
import shapely
import random
import pandas as pd

# Read the grid from a GPKG file
grid = gpd.read_file("china/map/filtered_grid/tree_grass_crops.gpkg").to_crs("EPSG:4326")

points = []
# Traverse each grid cell in the filtered GeoDataFrame
for idx, row in grid.iterrows():
    polygon = row['geometry']
    grid_id = row['id']
    # Generate a random point within the polygon
    minx, miny, maxx, maxy = polygon.bounds
    while True:
        random_point = shapely.geometry.Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if polygon.contains(random_point):
            break
    points.append({'id': grid_id, 'lon': random_point.x, 'lat': random_point.y})

# Save to CSV
points_df = pd.DataFrame(points)
points_df.to_csv("china/map/filtered_grid/random_points_in_filtered_grid_old.csv", index=False)

### Choose random points for China

In [ ]:
import rasterio.mask
import rasterio.transform
import numpy as np

# Paths to filtered grid and land cover raster files
grid_path = "china/map/filtered_grid/tree_grass_crops.gpkg"
dw_tif_path = "china/map/dw_china.tif"

grid = gpd.read_file(grid_path).to_crs("EPSG:4326")

points = []

# Open land cover raster
with rasterio.open(dw_tif_path) as src:
    for idx, row in grid.iterrows():
        polygon = row['geometry']
        grid_id = row['id']
        geom = [polygon]

        try: 
            out_image, out_transform = rasterio.mask.mask(src, geom, crop=True)
            label_pixels = out_image[0]
            valid_mask = label_pixels != src.nodata

            # find crop pixels (label : 4)
            crop_mask = (label_pixels == 4) & valid_mask 
            
            # If there are crop pixels in the grid cell, choose a random one
            if np.any(crop_mask):
                # Get indices of crop_pixels
                crop_indices = np.argwhere(crop_mask)
                # Choose a random crop pixel
                y, x = crop_indices[random.randint(0, len(crop_indices)-1)]
                # convert pixel indices to coordinates 
                lon, lat = rasterio.transform.xy(out_transform, y, x)
            
            else:
                # No crop region, pick random point in grid cell
                minx, miny, maxx, maxy = polygon.bounds 
                while True:
                    random_point = shapely.geometry.Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
                    if polygon.contains(random_point):
                        lon, lat = random_point.x, random_point.y
                        break
            points.append({'id': grid_id, 'lon': lon, 'lat': lat})
        
        except Exception as e:
            print(f"Failed on grid cell {grid_id} : {e}")

# Save to CSV
points_df = pd.DataFrame(points)
points_df.to_csv("china/map/filtered_grid/random_points_in_filtered_grid.csv", index=False)           
